In [ ]:
import geoplot as gplt #conda install -c conda-forge geoplot
import geoplot.crs as gcrs
import geopandas as gpd
gpd.__version__
import matplotlib.pyplot as plt
import pandas as pd
import pyproj
from fiona.crs import from_epsg
import mapclassify

In [ ]:
coastline= gpd.read_file('./Coastlinse_khoa/Coastlinse_khoa.shp', encoding='euc-kr')  
coastline.plot(figsize=(20,20))
coastline.crs

In [ ]:
data = pd.read_csv('&안전국어획고_2017_SJ_AREA.csv', encoding='euckr')
data.columns
data = data[['그룹','어업생산이', 'X_center', 'Y_center']]
geom_fish2 = gpd.points_from_xy(data.X_center, data.Y_center)
gdf_fish2 = gpd.GeoDataFrame(data, geometry=geom_fish2, crs=from_epsg(4326))
#df_cctv_gdf.info()
gdf_fish2 = gdf_fish2.to_crs(epsg=3857)
gdf_fish2.head()
# gdf_fish2.plot()

In [ ]:
coastline2 = coastline.to_crs(epsg=3857)
# coastline2.plot(color="goldenrod", edgecolor='k')

In [ ]:
gdf_fish2_4326 = gdf_fish2.to_crs(epsg=4326)
coastline2_4326 = coastline2.to_crs(epsg=4326)

In [ ]:
gal=pd.read_csv('전체대비_갈치류.csv',encoding='euckr')

In [ ]:
data = pd.read_csv('&안전국어획고_2017_SJ_AREA.csv', encoding='euckr')
data2= data[['그룹','어업생산이','MIN1', 'X_center', 'Y_center']]
grouped = data2.groupby('MIN1')
data2 = pd.DataFrame(grouped.mean()[['X_center', 'Y_center']])
data2.reset_index(inplace=True)
GRID_LIST = data['MIN1']

In [ ]:
match_df = pd.DataFrame()
GRID_LIST = gal['MIN1']
count=0
for GRID in GRID_LIST:
    print(len(GRID_LIST)-count)
    df2= data2[data2['MIN1']==GRID]
    match_df = match_df.append(df2[['MIN1', 'X_center', 'Y_center']])
    # print(df2[['X_center', 'Y_center']])
    count+=1
match = pd.merge(match_df, gal, on='MIN1', how='outer')

In [ ]:
gal

In [ ]:
match.dropna(inplace=True)
match

In [ ]:
geom_match = gpd.points_from_xy(match.X_center, match.Y_center)
gdf_match = gpd.GeoDataFrame(match[['MIN1','ratio']], geometry=geom_match, crs=from_epsg(4326))
gdf_match = gdf_match.to_crs(epsg=3857)
# gdf_match.head()
# gdf_match.plot()
gdf_match4326 = gdf_match.to_crs(epsg=4326)

In [ ]:
geom_match

In [ ]:
gdf_match4326

In [ ]:
gdf_match4326

In [ ]:
scheme = mapclassify.Quantiles(gdf_match4326['ratio'], k=10)
ax = gplt.pointplot(gdf_fish2_4326, color='powderblue',figsize=(50, 30), marker='s')

gplt.kdeplot(gdf_match4326, levels=50,common_norm=True, cmap='Reds',  figsize=(50, 30), alpha=0.5, shade=True,  ax = ax)

# gplt.kdeplot(gdf_match4326, hue='어업생산이', scheme=scheme,  cbar=True, figsize=(50, 30), shade=True,  ax = ax)
# # gplt.pointplot(gdf_match4326,hue ='어업생산이',scheme=scheme, legend = True, cmap='coolwarm',figsize=(50, 30), ax=ax, marker='s')
gplt.polyplot(coastline2_4326, facecolor='wheat', edgecolor='black',ax=ax, zorder=1,linewidth=0.5)

In [ ]:
for i in range(5):
    scheme = mapclassify.Quantiles(gdf_match4326['어업생산이'], k=10*(i+1))
    ax = gplt.pointplot(gdf_fish2_4326, color='ivory',figsize=(50, 30))
    gplt.pointplot(gdf_match4326,hue ='어업생산이',scheme=scheme, legend = True, cmap='jet',figsize=(50, 30), ax=ax, marker='s',
    legend_kwargs={'frameon': False, 'loc': 'lower right'})
    gplt.polyplot(coastline2_4326, facecolor='wheat', edgecolor='black',ax=ax, zorder=1,linewidth=0.5)
    plt.savefig('test_'+str(10*(i+1)))